In [2]:
#import libraries
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
#initialization of known entities
url = 'https://www.4icu.org/ph/'
db_name = 'university_rank_ph.db'
table_name = 'PH_Top_100_University'
csv_path = 'top-100-uni.csv'
df = pd.DataFrame(columns=["Rank","University","Town"])
count = 0

In [4]:
#load the webpage for webscraping
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

In [6]:
#scraping of information
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

In [7]:
#iterate over the contents of the rows
for row in rows:    
    #check for the loop counter to restrict to 50 entries
    if count<100:    
        #Extract all the td data objects in the row and save them to col
        col = row.find_all('td')    
        #Check if the length of col is 0
        if len(col)!=0: 
            #Create a dictionary data_dict with the keys same as the columns of the dataframe created
            data_dict = {"Rank": col[0].contents[0],
                         "University": col[1].contents[0],
                         "Town": col[2].contents[0]}
            #Convert the dictionary to a dataframe and concatenate it with the existing one
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            #Increment the loop counter
            count+=1
    else:
        break

In [8]:
print(df.head(10))

  Rank                                 University             Town
0    1      University of the Philippines Diliman  Quezon City ...
1    2                Ateneo de Manila University  Quezon City ...
2    3       University of the Philippines System      Quezon City
3    4                     De La Salle University       Manila ...
4    5                  University of Santo Tomas           Manila
5    6  Polytechnic University of the Philippines       Manila ...
6    7    University of the Philippines Los Baños        Los Baños
7    8       University of the Philippines Manila       Manila ...
8    9                   University of San Carlos        Cebu City
9   10                         University of Cebu    Cebu City ...


In [9]:
#save the dataframe to a csv file
df.to_csv(csv_path)

In [10]:
conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()